In [1]:
import pandas as pd
import numpy as np
import os
import cv2
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import VGG16
from tensorflow.keras import models, layers
from tensorflow.keras import layers, optimizers 
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.regularizers import l2
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from sklearn.preprocessing import LabelEncoder


2023-12-14 13:09:23.162597: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512_VNNI
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-14 13:09:23.338645: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
# Import data
class_names_dict = np.load("class_names.npy", allow_pickle=True).item()
attributes_npy = np.load("attributes.npy", allow_pickle=True)
attributes_df = pd.read_csv("attributes.txt", header=None, index_col = 0, sep=" |::", names=("attribute", "value"), engine='python')
training_labels_df = pd.read_csv("train_images.csv", header = 0)
test_images_paths = pd.read_csv("test_images_path.csv", header = 0)
test_images_sample = pd.read_csv("test_images_sample.csv", header = 0)

In [5]:
# local paths 
training_dir = "train_images_inferred_bad5/train_images" #This is the inferred augmented dir
testing_dir = "test_images" 

## Load Training

In [6]:
# This loads the training images from the balanced augmented training dir

images = []
labels = []

resize_dim = (224, 224)

# Iterate through subdirectories
for subdir in range(1, 201):
    subdir_path = os.path.join(training_dir, str(subdir))

    # Iterate through files in each subdirectory
    for filename in os.listdir(subdir_path):
        if filename.endswith(".jpg"):
            
            # Read images
            image_path = os.path.join(subdir_path, filename)
            img = cv2.imread(image_path)
            
            # Fix images
            img_resized = cv2.resize(img, resize_dim)
            img_normalized = img_resized / 255.0
            
            #Load images
            images.append(img_normalized)
            labels.append(subdir)

# Convert lists to numpy arrays
images_array = np.array(images)
labels_array = np.array(labels)


## Create Weight Dict (Not used any more)


In [1]:





# # Assuming 'labels' is your array of labels
# class_weights = compute_class_weight('balanced', classes=sorted(np.unique(labels_array)), y=labels)

# # Create a dictionary mapping class indices (starting from 0) to their corresponding weights
# weight_dict = dict(zip(range(len(class_weights)), class_weights))

# # Add an additional class and its weight
# additional_class_label = 200
# additional_class_weight = 3.926
# weight_dict[additional_class_label] = additional_class_weight



NameError: name 'compute_class_weight' is not defined

## Train Model

In [10]:
# Split to training and val set

train_birds_set, val_birds_set, train_labels_set, val_labels_set = train_test_split(images, labels, test_size=0.2, random_state=42)

In [11]:
# Convert sets to arrays because models dont accept lists

train_birds_set = np.array(train_birds_set)
val_birds_set = np.array(val_birds_set)
train_labels_set = np.array(train_labels_set)
val_labels_set = np.array(val_labels_set)

In [7]:

# Build MODEL
from tensorflow.keras.applications import DenseNet121 
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))  # Change this line

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

#Unfreeze some last layers
for layer in base_model.layers[-10:]:
    layer.trainable = True
    

def create_model():
    model = models.Sequential()
    model.add(base_model)
    
    
    model.add(layers.GlobalAveragePooling2D())
    #model.add(layers.Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
    #model.add(layers.Dropout(0.2))
    #model.add(layers.Dense(200, activation='softmax', kernel_regularizer=l2(0.01)))
    model.add(layers.Dense(256, activation='relu',))
    model.add(layers.Dense(201, activation='softmax', ))
              
    return model

# Create a fresh instance of the model
model = create_model()

2023-12-14 13:14:08.829384: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512_VNNI
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-14 13:14:09.249913: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38372 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:ca:00.0, compute capability: 8.0


In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               262400    
                                                                 
 dense_1 (Dense)             (None, 201)               51657     
                                                                 
Total params: 7,351,561
Trainable params: 482,185
Non-trainable params: 6,869,376
_________________________________________________________________


In [13]:
## Add more augmentation

datagen = ImageDataGenerator(

    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    vertical_flip=True,
)

datagen.fit(train_birds_set)

In [15]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


history = model.fit(train_birds_set, train_labels_set, epochs=10, 
                    validation_data=(val_birds_set, val_labels_set))

Epoch 1/10


/sw/arch/RHEL8/EB_production/2022/software/TensorFlow/2.11.0-foss-2022a-CUDA-11.7.0/lib/python3.10/site-packages/keras/backend.py:5585: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
2023-12-14 13:15:04.899393: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2023-12-14 13:15:08.293004: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-12-14 13:15:08.299252: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x152e4a4ffba0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-14 13:15:08.299276: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA A100-SXM4-40GB, Compute 

250/250 [==============================] - 25s 43ms/step - loss: 3.9868 - accuracy: 0.1746 - val_loss: 2.5480 - val_accuracy: 0.3900
Epoch 2/10
250/250 [==============================] - 6s 25ms/step - loss: 1.7680 - accuracy: 0.5636 - val_loss: 1.5662 - val_accuracy: 0.5855
Epoch 3/10
250/250 [==============================] - 6s 25ms/step - loss: 0.9773 - accuracy: 0.7475 - val_loss: 1.2083 - val_accuracy: 0.6745
Epoch 4/10
250/250 [==============================] - 6s 25ms/step - loss: 0.5743 - accuracy: 0.8549 - val_loss: 0.9538 - val_accuracy: 0.7495
Epoch 5/10
250/250 [==============================] - 6s 25ms/step - loss: 0.3482 - accuracy: 0.9183 - val_loss: 0.8281 - val_accuracy: 0.7820
Epoch 6/10
250/250 [==============================] - 6s 25ms/step - loss: 0.2332 - accuracy: 0.9481 - val_loss: 0.8203 - val_accuracy: 0.7945
Epoch 7/10
250/250 [==============================] - 6s 25ms/step - loss: 0.1532 - accuracy: 0.9679 - val_loss: 0.6956 - val_accuracy: 0.8285
Epoch 8/1

KeyboardInterrupt: 

In [23]:
# In case the above model is good SAVE IT !!

#model.save('best_model_densenet_43.h5')

In [57]:
#model = tf.keras.models.load_model('best_model_33.h5')

## Test Model

In [16]:
## Read images from test_dir (resize and rescale) in order and load test_images[] array


resize_dim = (224, 224)

def read_image(image_path):
    image = cv2.imread(testing_dir+image_path)
    img_resized = cv2.resize(image, resize_dim)
    img_normalized = img_resized / 255.0
    return img_normalized


test_images_paths['pixel_array'] = test_images_paths['image_path'].apply(read_image)

test_images = np.array(test_images_paths['pixel_array'].tolist())


In [17]:
predictions = model.predict(test_images)

125/125 [==============================] - 4s 19ms/step


In [18]:
predicted_labels = np.argmax(predictions, axis=1)

In [19]:
predicted_labels

array([ 69,  54,  74, ..., 135,  53, 111])

In [20]:
test_images_sample['label'] = predicted_labels

In [21]:
test_images_sample

,id,label
0,1,69
1,2,54
2,3,74
3,4,12
4,5,74
...,...,...
3995,3996,37
3996,3997,55
3997,3998,135
3998,3999,53


In [22]:
csv_file_path = 'kaggle_file.csv'

# Save the DataFrame to CSV
test_images_sample.to_csv(csv_file_path, index=False)

# Display a message indicating the file has been saved
print(f'DataFrame has been saved to {csv_file_path}')


DataFrame has been saved to kaggle_file.csv
